In [25]:
import findspark
findspark.init("/Users/chukuemekaogudu/Documents/Dev-Spark-Apache/Apache-Spark/spark-2.4.5-bin-hadoop2.7")

import os
import json
import time
from collections import Counter
from pyspark import SparkContext, SparkConf

In [26]:
data_dir = "/Volumes/oli2/inf533_datasets"

In [27]:
config = SparkConf().setMaster("local[*]") \
                    .setAppName("Task2Opt") \
                    .set("spark.executor.memory", "4g") \
                    .set("spark.driver.memory", "4g")
sc = SparkContext(conf=config).getOrCreate()

In [28]:
lines = sc.textFile(os.path.join(data_dir, "output.json")) \
          .map(json.loads) \
          .cache()

In [29]:
business_tokens = lines.filter(lambda x: x["description"] == "business_tokens")

In [30]:
businessID_tokens = business_tokens.map(lambda x: (x["id"], x["token"])) \
                                   .collectAsMap()

In [31]:
print(len(businessID_tokens))

10253


In [32]:
tokens_businessID = business_tokens.map(lambda x: (x["token"], x["id"])) \
                                   .collectAsMap()

In [33]:
count = 0
for k, v in tokens_businessID.items():
    if count == 10:
        break
    print(k, v)
    count += 1

0 bZMcorDrciRbjdjRyANcjA
1 n8Zqqhff-2cxzWt_nwhU2Q
2 1Df5WnLX3DqN6ymlhqznaQ
3 VfFHPsPtTW4Mgx0eHDyJiQ
4 bPcqucuuClxYrIM8xWoArg
5 6Z6IyosSMciZtwk8hRLRag
6 1xieLFUt_lgTUuGRabiMpQ
7 If6Bku2jkgPiikR6HBu-XQ
8 wqFkAsxYPA5tcdSkYMtrrw
9 aEGV0W2i8HRoDFsRrMz-BA


In [34]:
user_tokens = lines.filter(lambda x: x["description"] == "user_tokens")

In [35]:
userID_tokens = user_tokens.map(lambda x: (x["id"], x["token"])) \
                           .collectAsMap()

In [36]:
tokens_userID = user_tokens.map(lambda x: (x["token"], x["id"])) \
                           .collectAsMap()

In [37]:
print(len(userID_tokens))

26184


In [38]:
count = 0
for k, v in tokens_userID.items():
    if count == 10:
        break
    print(k, v)
    count += 1

0 txu_KwZOGYG6O3yYHjztbg
1 ELfzWgdf64VBLi5z1ECItw
2 rTIVsxvTEm5b94C2th5xQQ
3 PQ6-FZpUn2oFLyBGvdiG3w
4 TSMFB4bXjaEBdhQqIp8yGg
5 DeVGAiOf2mHVUDfxvuhVlQ
6 slazr8QlWY1Fuwrb3DJQ5Q
7 eMOpOLfR1Zz5tfHUlzMp_w
8 IoAWPFOjYUY-16q1mp-i1A
9 K2EV5TzIQRGn20WLpKLmcg


In [39]:
business_profile = lines.filter(lambda x: x["description"] == "business_profile") \
                        .map(lambda x: (x["id"], x["profile"])) \
                        .collectAsMap()

In [40]:
user_profile = lines.filter(lambda x: x["description"] == "user_profile") \
                    .map(lambda x: (x["id"], x["profile"])) \
                    .collectAsMap()

In [41]:
print(len(business_profile))
print(len(user_profile))

10253
26184


In [56]:
def cosine_similarity(s1, s2):
    import math
    denominator = len(s1) * len(s2)
    if denominator == 0:
        return 0.0
    denominator = math.sqrt(len(s1)) * math.sqrt(len(s2))
    return len(s1.intersection(s2)) / float(denominator)

In [67]:
prediction = sc.textFile(os.path.join(data_dir, "test_review.json")) \
               .map(json.loads) \
               .map(lambda x: ((x["user_id"], x["business_id"]), (userID_tokens.get(x["user_id"]), businessID_tokens.get(x["business_id"])))) \
               .filter(lambda x: x[1][0] != None and x[1][1] != None) \
               .mapValues(lambda x: cosine_similarity(set(user_profile[x[0]]), set(business_profile[x[1]]))) \
               .filter(lambda x: x[1] >= 0.01).collect()

In [70]:
value = prediction[0]
value[1]

0.35488672049383807

In [61]:
count = 0
for k, v in prediction.items():
    if count == 10:
        break
    print(k, v)
    count += 1

('mEzc6LeTNiQgIVsq3poMbg', '9UVkAmyodpaSBm49DBKFNw') 0.35488672049383807
('QtMgqKY_GF3XkOpXonaExA', 'IJUCRd5v-XLkcGrKjb8IfA') 0.3883099475736044
('nwvnNIixvyYTg4JS8g3Xgg', 'WQyGqfFKd-baBTVfZWzeTw') 0.2948913288139216
('aZtJzH3fRIRzrGnQRIVaRg', 'bKbYRUZKDYonSPOjzchJJg') 0.41360938972988415
('Vp5DzIiP_MmaMZMZVFFltA', 'ujHiaprwCQ5ewziu0Vi9rw') 0.29591817714964314
('srm0YUaJubOLxs4ByEZpwg', 'iCQpiavjjPzJ5_3gPD5Ebg') 0.27613264277890714
('XCNi6raOHuxmI66Cg2Er2Q', 'YJ8ljUhLsz6CtT_2ORNFmg') 0.29756216702269667
('QuZbJquRtbY9O9JrB9NpxQ', '6EWqsE6Ojvt-FA-rl8Sm4Q') 0.2813194041362935
('kwIhn1_cnQeUaLN0CuWWHw', 'TKcq0hOqesiDOwQb-ZwFAw') 0.3056839243938838
('7fyAtRSYYqOKZPr39L2UYg', 'uGupeWqih0yIcCg8anM1PA') 0.28823067684915676


In [62]:
ground_truth = sc.textFile(os.path.join(data_dir, "test_review_ratings.json")) \
                 .map(json.loads) \
                 .map(lambda x: ((x["user_id"], x["business_id"]), 0)) \
                 .collectAsMap()

In [63]:
for k in prediction.keys():
    ground_truth[k] += 1

In [64]:
print(len(ground_truth))

58480


In [65]:
print(sum(ground_truth.values()))

58433


In [66]:
58433/58480

0.9991963064295486